In [1]:
import pandas as pd
import numpy as np
import jieba
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
train = pd.read_csv("../input/mydata/cnews.train.txt", sep='\t', names=['label', 'content'])
x = train['content']
y = train['label']

In [2]:
#中文分词
def cut_content(data):
    words = data.apply(lambda x: ' '.join(jieba.cut(x)))
    return words
# 加载停用词表
stopwords_file = open('../input/myvocab/cnews.vocab.txt', encoding='utf-8')  
stopwords_list = stopwords_file.readlines()
stopwords = [x.strip() for x in stopwords_list]

In [3]:
accuracy=np.zeros(10)
precision_macro=np.zeros(10)
precision_micro=np.zeros(10)
recall_macro=np.zeros(10)
recall_micro=np.zeros(10)
f1_macro=np.zeros(10)
f1_micro=np.zeros(10)

In [4]:
kf = KFold(n_splits=10,shuffle=True,random_state=2021)
j=0
for train_index, test_index in kf.split(x):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train_cut = cut_content(x_train)
    cv = CountVectorizer(stop_words=stopwords, max_features=5000)
    cv_fit = cv.fit_transform(x_train_cut)
    train_matrix=cv_fit.toarray()
    label=['体育', '娱乐', '家居', '房产', '教育', '时尚', '时政', '游戏', '科技', '财经']
    p=np.ones((10,len(train_matrix[0])))
    pSum=np.ones(10)*2
    for i in range(len(train_matrix)):
        p[label.index(y_train.iloc[i])]+=train_matrix[i]
        pSum[label.index(y_train.iloc[i])]+=sum(train_matrix[i])
    for i in range(10):
        p[i]=np.log(p[i]/pSum[i])
    dict=cv.vocabulary_
    true_label=[]
    predict_label=[]
    for i in range(len(x_test)):
        x_test_cut = jieba.lcut(x_test.iloc[i])
        vector=np.zeros(len(dict))
        for each in x_test_cut:
            if each in dict:
                vector[dict[each]]=1
        pp=np.zeros(10)
        for ii in range(10):
            pp[ii]=sum(vector*p[ii])+np.log(0.1)
        pp=list(pp)
        true_label.append(y_test.iloc[i])
        predict_label.append(label[pp.index(max(pp))])
    accuracy[j]=accuracy_score(true_label, predict_label)
    precision_macro[j]=precision_score(true_label, predict_label, average='macro')
    precision_micro[j]=precision_score(true_label, predict_label, average='micro')
    recall_macro[j]=recall_score(true_label, predict_label, average='macro')
    recall_micro[j]=recall_score(true_label, predict_label, average='micro')
    f1_macro[j]=f1_score(true_label, predict_label, average='macro')
    f1_micro[j]=f1_score(true_label, predict_label, average='micro')
    j=j+1
print(sum(accuracy)/10,'\n')
print(sum(precision_macro)/10,'\n')
print(sum(precision_micro)/10,'\n')
print(sum(recall_macro)/10,'\n')
print(sum(recall_micro)/10,'\n')
print(sum(f1_macro)/10,'\n')
print(sum(f1_micro)/10,'\n')

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.273 seconds.
Prefix dict has been built successfully.
/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['pad'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


0.9404999999999999 

0.9420940088592461 

0.9404999999999999 

0.9405950968224575 

0.9404999999999999 

0.940564755481272 

0.9404999999999999 

